In [1]:
!python3 generate.py config/skyline.yaml ./ckpt/enc_dec_12L-16_bars-seqlen_1280/params/step_8100-RC_1.565-KL_3.176-model.pt generations/ 1 2

[preparing data] now at #0
[preparing data] now at #200
[preparing data] now at #400
[preparing data] now at #600
[preparing data] now at #800
[preparing data] now at #1000
[preparing data] now at #1200
[preparing data] now at #1400
[preparing data] now at #1600
[sampled pieces] [930]
/home/yihsin/miniforge3/envs/cne/lib/python3.10/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
Traceback (most recent call last):
  File "/home/yihsin/MidiStyleTransfer/MuseMorphose/generate.py", line 224, in <module>
    ).to(device)
  File "/home/yihsin/miniforge3/envs/cne/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1343, in to
    return self._apply(convert)
  File "/home/yihsin/miniforge3/envs/cne/lib/python3.10/site-packages/torch/nn/modules/module.py", line 903, in _apply
    m

#### tokenizer utils

In [25]:
import numpy as np
# define miditok tokenizer config
from miditok import REMI, TokenizerConfig
from symusic import Score

BEAT_RES = {(0, 1): 24, (1, 2): 8, (2, 4): 4, (4, 8): 2}
TOKENIZER_PARAMS = {
    "pitch_range": (21, 109),
    "beat_res": BEAT_RES,
    "num_velocities": 24,
    "special_tokens": ["PAD", "BOS", "EOS"],
    "use_chords": True,
    "use_rests": True,
    "use_tempos": True,
    "use_time_signatures": True,
    "use_programs": False,  # no multitrack here
    "num_tempos": 32,
    "tempo_range": (50, 200),  # (min_tempo, max_tempo)
}

# Creating a multitrack tokenizer, read the doc to explore all the parameters
config = TokenizerConfig(**TOKENIZER_PARAMS)
tokenizer = REMI(config)


def gnpy2midi(npy_path, midi_path="/content/test_from_npy.mid"):
  tokens = np.load(npy_path, allow_pickle=True)
  tokens = np.array([t for t in tokens if t<600])
  tokens = tokens.reshape(1, -1)
  #tokens = np.array([event2idx[int(e)] for e in tokens]).reshape(1,-1)
  converted_back_midi = tokenizer(tokens)
  print(converted_back_midi)
  converted_back_midi.dump_midi(midi_path) # Save the MIDI file

In [9]:
tokens = np.load("generations/id223_bar0_orig_orig.npy", allow_pickle=True)

In [7]:
import pickle
def read_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

dict_path = "pickles/skyline_miditok_vocab.pkl"
tokenizer_vocab = read_pickle(dict_path)
event2idx = tokenizer_vocab[0]                     # [{"event":idx}]
idx2event = [k for k, v in event2idx.items()]   # [event]

#### start converting generated result

In [27]:
gnpy2midi("generations/id1537_bar0_orig_orig.npy", "generations/sample01.mid")

Score(ttype=Tick, tpq=48, begin=0, end=2526, tracks=1, notes=328, time_sig=1, key_sig=0, markers=0)
